# Testing scipy.solve_bvp for the BVP Problem

This notebook demonstrates solving the second-order ODE:

$$-\Gamma R(r) = D(r) R''(r) + \left(\frac{D(r)}{r}+D'(r)\right) R'(r) - \frac{9 m^{2}}{4 r^{2}} D(r) R(r)$$

with boundary conditions $R'(0) = 0$ and $R'(1) = 0$.

In [17]:
import numpy as np
from scipy.integrate import solve_bvp
import matplotlib.pyplot as plt

# Set up plotting style
plt.rcParams['figure.figsize'] = (12, 8)

In [18]:
# Define D(r) and its derivative
def D(r):
    """Diffusion coefficient D(r) = (1/20)[1 + (1/2)cos²(πr)]"""
    return (1/20) * (1 + 0.5 * np.cos(np.pi * r)**2)

def D_prime(r):
    """Derivative of D(r)"""
    # D'(r) = (1/20) * (1/2) * 2 * cos(πr) * (-sin(πr)) * π
    return -(np.pi/20) * np.cos(np.pi * r) * np.sin(np.pi * r)

In [25]:
# Parameters
Gamma = 1.5  # Eigenvalue parameter (we'll solve for specific Gamma)
m = 1.0      # m parameter

In [26]:
# Convert to first-order system: y = [R, R']
# y[0] = R(r)
# y[1] = R'(r)
# Then: y[0]' = y[1]
#       y[1]' = R''(r)

def ode_system(r, y):
    """
    ODE system for solve_bvp
    Returns dy/dr = [y[1], y[1]']
    
    From the equation:
    -Γ R = D R'' + (D/r + D') R' - (9m²/4r²) D R
    
    Solving for R'':
    R'' = [-Γ R - (D/r + D') R' + (9m²/4r²) D R] / D
    """
    R = y[0]
    R_prime = y[1]
    
    Dr = D(r)
    Dpr = D_prime(r)
    
    # Handle singularity at r=0 (works with arrays)
    # Avoid division by zero by checking first
    r_safe = np.where(np.abs(r) < 1e-10, 1, r)  # Replace 0 with 1 temporarily
    
    coefficient = np.where(np.abs(r) < 1e-10, 
                          2 * Dpr,  # As r→0: D/r + D' → 2D'
                          Dr/r_safe + Dpr)
    
    centrifugal = np.where(np.abs(r) < 1e-10,
                          0,  # 1/r² term vanishes at r=0
                          (9 * m**2) / (4 * r_safe**2) * Dr)
    
    # R'' = [-Γ R - (D/r + D') R' + (9m²/4r²) D R] / D
    R_double_prime = (-Gamma * R - coefficient * R_prime + centrifugal * R) / Dr
    
    return np.vstack((R_prime, R_double_prime))

In [27]:
# Boundary conditions: R'(0) = 0 and R'(1) = 0
def boundary_conditions(ya, yb):
    """
    Boundary conditions at r=0 (ya) and r=1 (yb)
    ya[0] = R(0), ya[1] = R'(0)
    yb[0] = R(1), yb[1] = R'(1)
    
    We want: R'(0) = 0 and R'(1) = 0
    """
    return np.array([ya[1],  # R'(0) = 0
                     yb[1]])  # R'(1) = 0

In [28]:
# Initial mesh and guess
r_mesh = np.linspace(0, 1, 50)

# Initial guess: a simple function satisfying BCs (e.g., constant)
y_guess = np.zeros((2, r_mesh.size))
y_guess[0] = np.ones_like(r_mesh)  # R(r) = 1 (constant)
y_guess[1] = np.zeros_like(r_mesh)  # R'(r) = 0 (already satisfies BCs)

In [29]:
# Solve the BVP
print(f"Solving BVP with Γ = {Gamma}, m = {m}")
print(f"Boundary conditions: R'(0) = 0, R'(1) = 0")
print(f"D(r) = (1/20)[1 + (1/2)cos²(πr)]")
print("-" * 60)

solution = solve_bvp(ode_system, boundary_conditions, r_mesh, y_guess)

print(f"\nSolution status: {solution.message}")
print(f"Success: {solution.success}")
print(f"Number of mesh points: {solution.x.size}")
print(f"RMS residual: {solution.rms_residuals}")

Solving BVP with Γ = 1.5, m = 1.0
Boundary conditions: R'(0) = 0, R'(1) = 0
D(r) = (1/20)[1 + (1/2)cos²(πr)]
------------------------------------------------------------

Solution status: The algorithm converged to the desired accuracy.
Success: True
Number of mesh points: 50
RMS residual: [2.18941403e-12 8.85422628e-14 1.19856602e-14 1.03386133e-15
 1.82196011e-15 1.43515633e-16 1.07527861e-16 8.05869725e-17
 3.26895597e-16 2.10498700e-16 1.33252256e-16 4.72259975e-16
 1.96973229e-16 2.32922632e-16 2.03040906e-16 5.82584275e-17
 1.89429889e-16 1.10944130e-16 7.64802327e-17 1.12056415e-16
 2.37090713e-16 4.30391164e-17 2.68467610e-16 5.40594311e-17
 3.98217450e-18 3.94216293e-17 4.79951192e-19 3.24107160e-17
 9.26516068e-17 6.71672040e-17 1.20260664e-17 4.78263637e-17
 9.77034087e-17 1.05386414e-17 2.44037156e-17 9.07935759e-18
 1.04383528e-16 5.79205806e-17 6.40890251e-16 5.90267280e-16
 3.12183688e-16 1.11497481e-16 4.84799233e-16 5.39145296e-17
 2.27627250e-17 2.76471753e-17 2.70548

In [30]:
# Plot the solution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Dense mesh for smooth plotting
r_plot = np.linspace(0, 1, 200)
y_plot = solution.sol(r_plot)

# Plot R(r)
axes[0, 0].plot(r_plot, y_plot[0], 'b-', linewidth=2, label='R(r)')
axes[0, 0].set_xlabel('r', fontsize=12)
axes[0, 0].set_ylabel('R(r)', fontsize=12)
axes[0, 0].set_title('Solution R(r)', fontsize=14)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Plot R'(r)
axes[0, 1].plot(r_plot, y_plot[1], 'r-', linewidth=2, label="R'(r)")
axes[0, 1].set_xlabel('r', fontsize=12)
axes[0, 1].set_ylabel("R'(r)", fontsize=12)
axes[0, 1].set_title("Derivative R'(r)", fontsize=14)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[0, 1].legend()

# Plot D(r)
axes[1, 0].plot(r_plot, D(r_plot), 'g-', linewidth=2, label='D(r)')
axes[1, 0].set_xlabel('r', fontsize=12)
axes[1, 0].set_ylabel('D(r)', fontsize=12)
axes[1, 0].set_title('Diffusion Coefficient D(r)', fontsize=14)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# Verify the ODE residual
R = y_plot[0]
R_prime = y_plot[1]
Dr = D(r_plot)
Dpr = D_prime(r_plot)

# Calculate R'' numerically from the solution
R_double_prime = np.gradient(R_prime, r_plot)

# Calculate LHS and RHS of the original ODE
LHS = -Gamma * R
RHS = Dr * R_double_prime + (Dr/r_plot + Dpr) * R_prime - (9*m**2/(4*r_plot**2)) * Dr * R
# Handle r=0 singularity in RHS
RHS[0] = Dr[0] * R_double_prime[0] + 2*Dpr[0] * R_prime[0]

residual = LHS - RHS

axes[1, 1].plot(r_plot, residual, 'm-', linewidth=2, label='LHS - RHS')
axes[1, 1].set_xlabel('r', fontsize=12)
axes[1, 1].set_ylabel('Residual', fontsize=12)
axes[1, 1].set_title('ODE Residual (should be ≈ 0)', fontsize=14)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[1, 1].legend()
axes[1, 1].ticklabel_format(style='scientific', axis='y', scilimits=(-2, 2))

plt.tight_layout()
plt.show()

print(f"\nMax absolute residual: {np.max(np.abs(residual)):.2e}")

/var/folders/z4/h6nt6cb92jl9jsxg6g7h076m0000gp/T/ipykernel_12448/1348815391.py:44: RuntimeWarning: divide by zero encountered in divide
  RHS = Dr * R_double_prime + (Dr/r_plot + Dpr) * R_prime - (9*m**2/(4*r_plot**2)) * Dr * R
/var/folders/z4/h6nt6cb92jl9jsxg6g7h076m0000gp/T/ipykernel_12448/1348815391.py:44: RuntimeWarning: invalid value encountered in multiply
  RHS = Dr * R_double_prime + (Dr/r_plot + Dpr) * R_prime - (9*m**2/(4*r_plot**2)) * Dr * R



Max absolute residual: 1.52e-13


In [ ]:
# Verify boundary conditions
print("\nBoundary Condition Verification:")
print(f"R'(0) = {y_plot[1][0]:.6e} (should be ≈ 0)")
print(f"R'(1) = {y_plot[1][-1]:.6e} (should be ≈ 0)")
print(f"\nR(0) = {y_plot[0][0]:.6f}")
print(f"R(1) = {y_plot[0][-1]:.6f}")

In [ ]:
# Plot the first few eigenfunctions
if len(peaks) > 0:
    n_eigenfunctions = min(4, len(peaks))
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    for i in range(n_eigenfunctions):
        if i < len(peaks):
            Gamma = gamma_fine[peaks[i]]
            
            # Solve for this eigenvalue
            sol = solve_bvp(ode_system, boundary_conditions, r_mesh, y_guess, verbose=0)
            
            if sol.success:
                r_dense = np.linspace(0, 1, 200)
                y_dense = sol.sol(r_dense)
                
                # Normalize the eigenfunction
                norm = np.sqrt(np.trapz(y_dense[0]**2, r_dense))
                R_normalized = y_dense[0] / norm if norm > 1e-10 else y_dense[0]
                
                axes[i].plot(r_dense, R_normalized, 'b-', linewidth=2)
                axes[i].set_xlabel('r', fontsize=12)
                axes[i].set_ylabel('R(r) (normalized)', fontsize=12)
                axes[i].set_title(f'Eigenfunction {i+1}: Γ = {Gamma:.4f}', fontsize=14)
                axes[i].grid(True, alpha=0.3)
                axes[i].axhline(y=0, color='k', linestyle='--', alpha=0.3)
    
    # Hide unused subplots
    for i in range(n_eigenfunctions, 4):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No eigenvalues found to plot eigenfunctions.")

In [ ]:
# Better approach: Use a finer scan and look for peaks in solution norm
gamma_fine = np.linspace(0.01, 5, 500)
solution_norms = []

print("Fine scanning for eigenvalues...")

for gamma_test in gamma_fine:
    Gamma = gamma_test
    
    try:
        sol = solve_bvp(ode_system, boundary_conditions, r_mesh, y_guess, verbose=0)
        if sol.success:
            # Calculate L2 norm of solution
            r_dense = np.linspace(0, 1, 100)
            y_dense = sol.sol(r_dense)
            norm = np.sqrt(np.trapz(y_dense[0]**2, r_dense))
            solution_norms.append(norm)
        else:
            solution_norms.append(0)
    except:
        solution_norms.append(0)

solution_norms = np.array(solution_norms)

# Find peaks (eigenvalues)
from scipy.signal import find_peaks

peaks, properties = find_peaks(solution_norms, height=0.1, prominence=0.05)

plt.figure(figsize=(14, 6))
plt.plot(gamma_fine, solution_norms, 'b-', linewidth=2, label='||R||₂')
if len(peaks) > 0:
    plt.plot(gamma_fine[peaks], solution_norms[peaks], 'ro', markersize=10, 
             label=f'Eigenvalues (n={len(peaks)})')
plt.xlabel('Γ', fontsize=14)
plt.ylabel('||R(r)||₂ (L² norm)', fontsize=14)
plt.title('Solution Norm vs Γ - Eigenvalue Detection', fontsize=16)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

print(f"\nFound {len(peaks)} potential eigenvalues:")
for i, peak_idx in enumerate(peaks[:10]):
    print(f"  Γ_{i+1} ≈ {gamma_fine[peak_idx]:.6f}, ||R||₂ = {solution_norms[peak_idx]:.4e}")

### Alternative: Direct Eigenvalue Problem

For a more robust approach, we can reformulate this as a standard eigenvalue problem. Since the trivial solution $R=0$ always exists, we need to find $\Gamma$ values where the determinant of the system vanishes (i.e., where non-trivial solutions appear).

In [ ]:
# Scan different values of Gamma to find eigenvalues
# For an eigenvalue, we should get a non-trivial solution where R is not identically zero

gamma_values = np.linspace(0.1, 10, 100)
max_R_values = []
success_flags = []

print("Scanning for eigenvalues...")
print("-" * 60)

for gamma_test in gamma_values:
    Gamma = gamma_test
    
    # Try to solve with this Gamma
    try:
        sol = solve_bvp(ode_system, boundary_conditions, r_mesh, y_guess, verbose=0)
        if sol.success:
            # Evaluate solution
            y_test = sol.sol(r_mesh)
            max_R = np.max(np.abs(y_test[0]))
            max_R_values.append(max_R)
            success_flags.append(True)
        else:
            max_R_values.append(0)
            success_flags.append(False)
    except:
        max_R_values.append(0)
        success_flags.append(False)

max_R_values = np.array(max_R_values)
success_flags = np.array(success_flags)

# Plot the maximum |R| as a function of Gamma
plt.figure(figsize=(12, 6))
plt.semilogy(gamma_values[success_flags], max_R_values[success_flags], 'b-', linewidth=2)
plt.xlabel('Γ', fontsize=14)
plt.ylabel('max|R(r)|', fontsize=14)
plt.title('Solution Magnitude vs Γ (Eigenvalue Search)', fontsize=16)
plt.grid(True, alpha=0.3)
plt.axhline(y=1e-6, color='r', linestyle='--', label='Trivial threshold')
plt.legend()
plt.tight_layout()
plt.show()

# Identify potential eigenvalues (where solution is non-trivial)
eigenvalue_threshold = 0.01
potential_eigenvalues = gamma_values[(max_R_values > eigenvalue_threshold) & success_flags]

print(f"\nPotential eigenvalues (Γ where max|R| > {eigenvalue_threshold}):")
if len(potential_eigenvalues) > 0:
    for i, gamma_ev in enumerate(potential_eigenvalues[:10]):  # Show first 10
        idx = np.where(gamma_values == gamma_ev)[0][0]
        print(f"  Γ ≈ {gamma_ev:.4f}, max|R| = {max_R_values[idx]:.4e}")
else:
    print("  No clear eigenvalues found in this range.")

## Finding Eigenvalues

Since this is an eigenvalue problem with homogeneous boundary conditions, only discrete values of $\Gamma$ (eigenvalues) will yield non-trivial solutions. Let's scan through different values of $\Gamma$ to find which ones give non-trivial solutions.